# Demo 2: Darwinian Evolution of Code

## Concept: Genetic Algorithm with LLM as Mutation Operator

In this demo, we move beyond fixing errors to **optimizing solutions**. We'll use evolutionary principles:

```
Population → Evaluation → Selection → Mutation/Crossover → New Population
```

### What We'll Build

A system that evolves passwords meeting complex, arbitrary rules:
- Specific length requirements
- Must contain emojis 🔐
- Digits must sum to exactly 15
- And more!

### The Key Innovation

The LLM acts as the **mutation/crossover operator**. Instead of random bit flips, we get *intelligent* recombination that understands what makes a good solution.

---

### Related Papers

- **FunSearch**: Making new discoveries in mathematical sciences using Large Language Models  
  [Nature (2023)](https://www.nature.com/articles/s41586-023-06924-6)

- **Eureka**: Human-Level Reward Design via Coding Large Language Models  
  [arXiv:2310.12931](https://arxiv.org/abs/2310.12931)

## Setup

You have three options for the LLM provider:

### Option 1: OpenAI (Recommended)
1. Get an API key from [platform.openai.com](https://platform.openai.com)
2. Add secret `OPENAI_API_KEY` in Colab Secrets (key icon in sidebar)

### Option 2: Google Gemini (FREE)
1. Get a free API key from [Google AI Studio](https://aistudio.google.com/apikey)
2. Add secret `GEMINI_API_KEY` in Colab Secrets
3. In the next cell, comment out the OpenAI section and uncomment the Gemini section

### Option 3: Groq (FREE - Very Fast)
1. Get a free API key from [console.groq.com](https://console.groq.com)
2. Add secret `GROQ_API_KEY` in Colab Secrets
3. In the next cell, comment out the OpenAI section and uncomment the Groq section
4. Uses Llama 3.1 model

In [1]:
# ============================================================
# OPTION 1: OpenAI (Recommended - requires API key with credits)
# ============================================================
!pip install openai -q

from google.colab import userdata
from openai import OpenAI
import re

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

print("Setup complete! Using OpenAI")

# ============================================================
# OPTION 2: Google Gemini (FREE - uncomment below, comment above)
# ============================================================
# !pip install google-generativeai -q
#
# from google.colab import userdata
# import google.generativeai as genai
# import re
#
# genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
#
# # Wrapper class to make Gemini API compatible with OpenAI-style calls
# class GeminiClient:
#     def __init__(self):
#         self._model = genai.GenerativeModel('gemini-1.5-flash')
#
#     class _Completions:
#         def __init__(self, model):
#             self._model = model
#
#         def create(self, model=None, messages=None, temperature=0.7, **kwargs):
#             prompt_parts = []
#             for msg in messages:
#                 role = msg.get('role', 'user')
#                 content = msg.get('content', '')
#                 if role == 'system':
#                     prompt_parts.append(f"Instructions: {content}")
#                 else:
#                     prompt_parts.append(content)
#
#             prompt = "\n\n".join(prompt_parts)
#
#             response = self._model.generate_content(
#                 prompt,
#                 generation_config=genai.GenerationConfig(temperature=temperature)
#             )
#
#             class Message:
#                 def __init__(self, text):
#                     self.content = text
#
#             class Choice:
#                 def __init__(self, text):
#                     self.message = Message(text)
#
#             class Response:
#                 def __init__(self, text):
#                     self.choices = [Choice(text)]
#
#             return Response(response.text)
#
#     @property
#     def chat(self):
#         class Chat:
#             def __init__(chat_self):
#                 chat_self.completions = GeminiClient._Completions(self._model)
#         return Chat()
#
# client = GeminiClient()
#
# print("Setup complete! Using Google Gemini (FREE)")

# ============================================================
# OPTION 3: Groq (FREE - very fast, uncomment below, comment above)
# ============================================================
# !pip install openai -q
#
# from google.colab import userdata
# from openai import OpenAI
# import re
#
# client = OpenAI(
#     api_key=userdata.get('GROQ_API_KEY'),
#     base_url="https://api.groq.com/openai/v1"
# )
#
# # IMPORTANT: When using Groq, change the model in API calls from
# # "gpt-4o-mini" to "llama-3.1-8b-instant"
#
# print("Setup complete! Using Groq (FREE)")

Setup complete! Using OpenAI


## The Problem: Complex Password Generation

We want to generate passwords that meet **all** of these criteria:

| Rule | Points | Description |
|------|--------|-------------|
| Length | 20 pts | Between 12-20 characters |
| Digits | 15 pts | At least 2 digits |
| Case Mix | 15 pts | Both uppercase and lowercase |
| Emoji | 20 pts | At least 1 emoji |
| Special | 15 pts | Contains !@#$%^&* |
| Digit Sum | 15 pts | Digits sum to exactly 15 |

**Maximum score: 100 points**

This is hard because the rules interact! Adding digits to reach sum=15 might push you over the length limit.

## The Fitness Function

In evolutionary algorithms, the **fitness function** determines how "good" each candidate is. It's the selection pressure that drives evolution.

Our fitness function scores each password from 0-100 based on how many rules it satisfies.

In [2]:
def evaluate_password(password: str) -> dict:
    """
    Fitness function: Score a password based on complex rules.

    Args:
        password: The password string to evaluate

    Returns:
        Dict with 'password', 'score' (0-100), and 'criteria' breakdown
    """
    score = 0
    criteria = {}

    # Rule 1: Length between 12-20 chars (20 points)
    if 12 <= len(password) <= 20:
        criteria["length"] = 20
        score += 20
    else:
        criteria["length"] = 0

    # Rule 2: Contains at least 2 digits (15 points)
    digits = sum(c.isdigit() for c in password)
    if digits >= 2:
        criteria["digits"] = 15
        score += 15
    else:
        criteria["digits"] = 0

    # Rule 3: Contains uppercase and lowercase (15 points)
    has_upper = any(c.isupper() for c in password)
    has_lower = any(c.islower() for c in password)
    if has_upper and has_lower:
        criteria["case_mix"] = 15
        score += 15
    else:
        criteria["case_mix"] = 0

    # Rule 4: Contains at least 1 emoji (20 points)
    emoji_pattern = re.compile("[\U0001F300-\U0001F9FF]")
    if emoji_pattern.search(password):
        criteria["emoji"] = 20
        score += 20
    else:
        criteria["emoji"] = 0

    # Rule 5: Contains special characters !@#$%^&* (15 points)
    special = set("!@#$%^&*")
    if any(c in special for c in password):
        criteria["special"] = 15
        score += 15
    else:
        criteria["special"] = 0

    # Rule 6: Digits sum to exactly 15 (15 points)
    digit_sum = sum(int(c) for c in password if c.isdigit())
    if digit_sum == 15:
        criteria["digit_sum_15"] = 15
        score += 15
    else:
        criteria["digit_sum_15"] = 0

    return {"password": password, "score": score, "criteria": criteria, "digit_sum": digit_sum}


# Test the fitness function
print("🧪 Testing fitness function:\n")
test_passwords = [
    "simple",           # Too short, no criteria met
    "Password123!",     # Good but no emoji, digits sum wrong
    "Pass96!🔐",        # Has emoji! Let's see...
]

for pw in test_passwords:
    result = evaluate_password(pw)
    print(f"Password: {result['password']}")
    print(f"Score: {result['score']}/100 (digit sum: {result['digit_sum']})")
    print(f"Criteria: {result['criteria']}\n")

🧪 Testing fitness function:

Password: simple
Score: 0/100 (digit sum: 0)
Criteria: {'length': 0, 'digits': 0, 'case_mix': 0, 'emoji': 0, 'special': 0, 'digit_sum_15': 0}

Password: Password123!
Score: 65/100 (digit sum: 6)
Criteria: {'length': 20, 'digits': 15, 'case_mix': 15, 'emoji': 0, 'special': 15, 'digit_sum_15': 0}

Password: Pass96!🔐
Score: 80/100 (digit sum: 15)
Criteria: {'length': 0, 'digits': 15, 'case_mix': 15, 'emoji': 20, 'special': 15, 'digit_sum_15': 15}



## Generation 0: Initial Population

Every evolutionary algorithm needs an initial population. We'll ask the LLM to generate 5 initial password candidates.

Note: This first generation is essentially random exploration. The LLM will try, but probably won't hit 100/100 on the first try.

In [3]:
def generate_initial_population(n: int = 5) -> list:
    """
    Generate n password candidates using LLM.

    Args:
        n: Number of passwords to generate

    Returns:
        List of password strings
    """
    prompt = f"""Generate {n} creative password candidates that try to meet these criteria:
    - Length between 12-20 characters
    - At least 2 digits
    - Mix of uppercase and lowercase letters
    - At least 1 emoji (like 🔐, 🚀, 💪, etc.)
    - Special characters (!@#$%^&*)
    - Digits should sum to exactly 15 (e.g., 9+6=15, or 8+7=15, or 5+5+5=15)

    Return ONLY the {n} passwords, one per line, nothing else."""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9  # High temperature for diversity
    )

    passwords = response.choices[0].message.content.strip().split("\n")
    # Clean up and limit to n passwords
    return [p.strip() for p in passwords if p.strip()][:n]

In [4]:
# Generate and evaluate initial population
print("🧬 Generation 0: Initial Population")
print("=" * 50)
print()

population = generate_initial_population(5)
evaluated = [evaluate_password(p) for p in population]

# Sort by score (best first)
evaluated.sort(key=lambda x: x["score"], reverse=True)

for i, item in enumerate(evaluated):
    print(f"{i+1}. Score: {item['score']}/100 - {item['password']}")
    print(f"   Digit sum: {item['digit_sum']} (target: 15)")
    print(f"   Criteria: {item['criteria']}")
    print()

🧬 Generation 0: Initial Population

1. Score: 85/100 - 1. 🌟Rocket123!9&6
   Digit sum: 22 (target: 15)
   Criteria: {'length': 20, 'digits': 15, 'case_mix': 15, 'emoji': 20, 'special': 15, 'digit_sum_15': 0}

2. Score: 85/100 - 2. 💪Strong4*5*6*0
   Digit sum: 17 (target: 15)
   Criteria: {'length': 20, 'digits': 15, 'case_mix': 15, 'emoji': 20, 'special': 15, 'digit_sum_15': 0}

3. Score: 85/100 - 4. 🔐Protect8*6*1#
   Digit sum: 19 (target: 15)
   Criteria: {'length': 20, 'digits': 15, 'case_mix': 15, 'emoji': 20, 'special': 15, 'digit_sum_15': 0}

4. Score: 85/100 - 5. Magic2&2*4@7🔑
   Digit sum: 20 (target: 15)
   Criteria: {'length': 20, 'digits': 15, 'case_mix': 15, 'emoji': 20, 'special': 15, 'digit_sum_15': 0}

5. Score: 65/100 - 3. Secure5#7@3!0
   Digit sum: 18 (target: 15)
   Criteria: {'length': 20, 'digits': 15, 'case_mix': 15, 'emoji': 0, 'special': 15, 'digit_sum_15': 0}



## Selection & Mutation via LLM

This is where the magic happens! Traditional genetic algorithms use:
- **Selection**: Keep the fittest individuals
- **Crossover**: Combine traits from two parents
- **Mutation**: Random changes to explore new solutions

We'll use the **LLM as an intelligent mutation/crossover operator**:
- It sees the best parents and their scores
- It understands which criteria are missing
- It creates new candidates that combine the best traits

In [5]:
def evolve_population(parents: list, generation: int) -> list:
    """
    Take the 2 best parents and ask LLM to create mutations/crossovers.

    Args:
        parents: List of evaluated password dicts (sorted by score)
        generation: Current generation number (for logging)

    Returns:
        List of new password strings
    """
    parent1 = parents[0]["password"]
    parent2 = parents[1]["password"]
    p1_score = parents[0]["score"]
    p2_score = parents[1]["score"]
    p1_criteria = parents[0]["criteria"]
    p2_criteria = parents[1]["criteria"]
    p1_digit_sum = parents[0]["digit_sum"]
    p2_digit_sum = parents[1]["digit_sum"]

    prompt = f"""You are evolving passwords through a genetic algorithm. Generation {generation}.

    PARENT 1 (Score: {p1_score}/100): {parent1}
    - Criteria met: {p1_criteria}
    - Current digit sum: {p1_digit_sum} (target: 15)

    PARENT 2 (Score: {p2_score}/100): {parent2}
    - Criteria met: {p2_criteria}
    - Current digit sum: {p2_digit_sum} (target: 15)

    TARGET CRITERIA (to maximize score to 100):
    - Length 12-20 chars (20 pts)
    - At least 2 digits (15 pts)
    - Mix upper/lowercase (15 pts)
    - At least 1 emoji (20 pts)
    - Special chars !@#$%^&* (15 pts)
    - Digits sum to EXACTLY 15 (15 pts) - e.g., 9+6=15, 8+7=15, 5+5+5=15

    Create 5 NEW passwords by combining and mutating the best traits from both parents.
    Focus especially on:
    1. Getting the digit sum to exactly 15
    2. Improving any criteria that parents are missing
    3. Keeping the traits that are already working

    Return ONLY 5 passwords, one per line, nothing else."""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8  # Slightly lower for more focused improvement
    )

    children = response.choices[0].message.content.strip().split("\n")
    return [c.strip() for c in children if c.strip()][:5]

## The Evolution Loop

Now let's run evolution for 3 generations and watch the population improve!

Each generation:
1. Select the 2 best parents
2. Ask LLM to create 5 children through crossover/mutation
3. Evaluate and rank the new generation
4. Repeat

In [8]:
NUM_GENERATIONS = 10

# Track best scores across generations for visualization
best_scores = [evaluated[0]["score"]]

for gen in range(1, NUM_GENERATIONS + 1):
    print(f"\n{'='*60}")
    print(f"🧬 Generation {gen}: Evolution")
    print(f"{'='*60}\n")

    # Select top 2 parents
    top_parents = evaluated[:2]
    print(f"👨‍👩‍👧‍👦 Parents selected:")
    print(f"  1. {top_parents[0]['password']}")
    print(f"     Score: {top_parents[0]['score']}/100, Digit sum: {top_parents[0]['digit_sum']}")
    print(f"  2. {top_parents[1]['password']}")
    print(f"     Score: {top_parents[1]['score']}/100, Digit sum: {top_parents[1]['digit_sum']}")
    print()

    # Generate children through crossover/mutation
    print("🔄 Generating offspring via LLM crossover/mutation...\n")
    children = evolve_population(top_parents, gen)

    # Evaluate new generation
    evaluated = [evaluate_password(p) for p in children]
    evaluated.sort(key=lambda x: x["score"], reverse=True)

    # Track best score
    best_scores.append(evaluated[0]["score"])

    print("🌱 New Generation:")
    for i, item in enumerate(evaluated):
        status = "🏆" if item["score"] == 100 else "  "
        print(f"{status} {i+1}. Score: {item['score']}/100 - {item['password']}")
        print(f"      Digit sum: {item['digit_sum']} (target: 15)")

    # Check if we found a perfect solution
    if evaluated[0]["score"] == 100:
        print("\n🎉 PERFECT SCORE ACHIEVED!")
        break


🧬 Generation 1: Evolution

👨‍👩‍👧‍👦 Parents selected:
  1. 1. 🌟Rocket4*3*5*3
     Score: 85/100, Digit sum: 16
  2. 2. 💪Strong5*2*4*4
     Score: 85/100, Digit sum: 17

🔄 Generating offspring via LLM crossover/mutation...

🌱 New Generation:
   1. Score: 85/100 - 1. 🌟Rocket4*2*3*6
      Digit sum: 16 (target: 15)
   2. Score: 85/100 - 2. 💪Strong5*2*3*5
      Digit sum: 17 (target: 15)
   3. Score: 85/100 - 3. 🚀Power3*3*4*5
      Digit sum: 18 (target: 15)
   4. Score: 85/100 - 4. 🌟Strength7*2*3*3
      Digit sum: 19 (target: 15)
   5. Score: 85/100 - 5. 💪Mighty4*1*5*5
      Digit sum: 20 (target: 15)

🧬 Generation 2: Evolution

👨‍👩‍👧‍👦 Parents selected:
  1. 1. 🌟Rocket4*2*3*6
     Score: 85/100, Digit sum: 16
  2. 2. 💪Strong5*2*3*5
     Score: 85/100, Digit sum: 17

🔄 Generating offspring via LLM crossover/mutation...

🌱 New Generation:
🏆 1. Score: 100/100 - 1. 🌟Rocket4*2*3*5
      Digit sum: 15 (target: 15)
   2. Score: 85/100 - 2. 💪Strong3*2*3*7
      Digit sum: 17 (target: 15)
   3. 

## Final Results

Let's see the best password we evolved and how scores improved across generations.

In [9]:
print(f"{'='*60}")
print(f"🏆 EVOLUTION COMPLETE")
print(f"{'='*60}")
print()

best = evaluated[0]
print(f"Best Password: {best['password']}")
print(f"Final Score: {best['score']}/100")
print(f"Digit Sum: {best['digit_sum']} (target: 15)")
print()
print("Criteria Breakdown:")
for criterion, points in best['criteria'].items():
    status = "✅" if points > 0 else "❌"
    print(f"  {status} {criterion}: {points} pts")

print()
print("📈 Score progression across generations:")
for i, score in enumerate(best_scores):
    bar = "█" * (score // 5)
    print(f"  Gen {i}: {bar} {score}/100")

🏆 EVOLUTION COMPLETE

Best Password: 1. 🌟Rocket4*2*3*5
Final Score: 100/100
Digit Sum: 15 (target: 15)

Criteria Breakdown:
  ✅ length: 20 pts
  ✅ digits: 15 pts
  ✅ case_mix: 15 pts
  ✅ emoji: 20 pts
  ✅ special: 15 pts
  ✅ digit_sum_15: 15 pts

📈 Score progression across generations:
  Gen 0: █████████████████ 85/100
  Gen 1: █████████████████ 85/100
  Gen 2: ████████████████████ 100/100


## Key Takeaways

### What We Learned

1. **LLM as Intelligent Operator**: Unlike random mutation, the LLM understands the problem and makes *directed* improvements. It can reason about which traits to keep and which to change.

2. **Fitness-Guided Search**: The fitness function provides the "selection pressure" that guides evolution. Without it, we'd just have random generation.

3. **Exploration vs Exploitation**:
   - High temperature = more exploration (diverse but unpredictable)
   - Low temperature = more exploitation (focused but may get stuck)

4. **Emergent Solutions**: The final password may have creative solutions we didn't explicitly design for!

### Real-World Applications

This technique is used for:
- **FunSearch (DeepMind)**: Discovering new mathematical algorithms
- **Eureka (NVIDIA)**: Evolving reward functions for robot training
- **Prompt optimization**: Finding the best prompts for LLMs
- **Code optimization**: Evolving more efficient algorithms

### Next Steps

In Demo 3, we'll see how an agent can **create entirely new tools** for itself - going beyond optimization to true self-modification!